# DS 300 Mini Project 1

## Joseph Sepich and Hunter Dicicco

### Setup and Imports

In [1]:
import pickle

from subprocess import run
from hashlib import sha256

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline
plt.style.use('seaborn-whitegrid')

random_state = 26498

### Read in Data

In [2]:
airbnb_data = pd.read_csv("AB_NYC_2019.csv")
airbnb_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


# Dataset Analysis

Directions:
````
List the column names in the dataset. For each column, identify the type of information (string, integer, etc) and present the basic descriptives (range of values, mean, frequencies, outliers or unique values, etc). Explain the utility of the data that you want to preserve after your anonymization process. For example, discuss what types of queries you want your anonymized data to support. Your anonymization approach should support your answer.
````

### Column Description

Note that in this data set, each row or case represents a property listed for rent.

In [3]:
# Get mean stats
airbnb_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [4]:
# Cardinality
print("="*13 + "\nCardinalities\n" + "="*13 + "\n")
for column in airbnb_data.columns:
    print(f"{column:{len(max(airbnb_data.columns, key=len)) + 1}}: {airbnb_data[column].nunique()} unique values")

Cardinalities

id                             : 48895 unique values
name                           : 47905 unique values
host_id                        : 37457 unique values
host_name                      : 11452 unique values
neighbourhood_group            : 5 unique values
neighbourhood                  : 221 unique values
latitude                       : 19048 unique values
longitude                      : 14718 unique values
room_type                      : 3 unique values
price                          : 674 unique values
minimum_nights                 : 109 unique values
number_of_reviews              : 394 unique values
last_review                    : 1764 unique values
reviews_per_month              : 937 unique values
calculated_host_listings_count : 47 unique values
availability_365               : 366 unique values


In [5]:
# Range
range_cols = ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
print("="*6 + "\nRanges\n" + "="*6 + "\n")
for col in range_cols:
    print(f"{col:{len(max(range_cols, key=len)) + 1}}: {airbnb_data[col].max() - airbnb_data[col].min()}")

Ranges

latitude                       : 0.41326999999999003
longitude                      : 0.5314299999999861
price                          : 10000
minimum_nights                 : 1249
number_of_reviews              : 629
reviews_per_month              : 58.49
calculated_host_listings_count : 326
availability_365               : 365


| Column Name | Data Type | Mean | Range | Cardinality | Description |
|------------:|-----------|------|-------|-------------|-------------|
|     Id      | Integer   |      |       |48895|Listing Identifier|
|    host_id  | Integer   |      |       |37457|Host Identifier|
|latitude     | Double    |40.72849|0.41326999|19048|Latitude Location|
|longitude    | Double    |-73.952170|0.5314299|14718|Longitude Location|
|price        | Integer   |152.720687|10000|674|Nightly Rental Rate|
|minimum_nights| Integer  |7.029962|1249|109|Rental Night Minimum|
|number_of_reviews|Integer|23.274466|629|394|Total Reviews for Property|
|reviews_per_month|Double |1.373221|58.49|937|Reviews each Month|
|host_listings_count|Integer|7.143982|326|47|Total Listings for host|
|availability_365|Integer |112.781327|365|366|Days available in the year|
|neighbourhood_group|String|      |       |5|Big Five Neighbourhood Location|
|neighbourhood|String     |      |       |221|Sub Neighbourhood Location|
|name         |String     |      |       |47905|Name of Rental|
|host_name    |String     |      |       |11452|Name of Host|
|room_type    |String     |      |       |3|Type of Rental available|
|last_review  |Date     |      |       |1764|Date of Last review|



### Utility Preservation Goals

In our anonymization approach we want to **preserve queries** that relate to **aggregate statistics**.

This is especially true for maintaining the data's distribution. If we can keep the distribution, then these types of queries will be more accurate. We also want to keep some fairly generic information available like type of the room or how long you have to rent there. Some example queries that we would want to preserve are:

* SELECT mean(price) FROM airbnb_listings
* SELECT mean(price) FROM airbnb_listings WHERE minimum_nights < 3
* SELECT min(price), max(price) FROM airbnb_listings WHERE neighbourhood_group == "Manhattan"
* SELECT min(price), max(price) FROM airbnb_listings WHERE room_type == "Entire home/apt"
* SELECT room_type, minimum_nights FROM airbnb_listings WHERE neighbourhood_group == "Brooklyn"


In [6]:
print("="*12 + f"\nMean price from all listings: ${airbnb_data['price'].mean():.{5}}\n" + "="*12)
print(f"Mean price from listings with 3 nights minimum: ${airbnb_data[airbnb_data['minimum_nights'] < 3]['price'].mean():.{5}}\n" + "="*12)
print(f"Price range from Manhattan neighborhood rentals: [${airbnb_data[airbnb_data['neighbourhood_group'] == 'Manhattan']['price'].min()}, ${airbnb_data[airbnb_data['neighbourhood_group'] == 'Manhattan']['price'].max()}]\n" + "="*12)
print(f"Price range for Entire Home rentals: [${airbnb_data[airbnb_data['room_type'] == 'Entire home/apt']['price'].min()}, ${airbnb_data[airbnb_data['room_type'] == 'Entire home/apt']['price'].max()}]\n" + "="*12)

print("Room Type and Minimum Nights from Brooklyn Rentals:")
print(airbnb_data[airbnb_data['neighbourhood_group'] == 'Brooklyn'][['room_type', 'minimum_nights']].sample(n=10, random_state=random_state))

Mean price from all listings: $152.72
Mean price from listings with 3 nights minimum: $144.06
Price range from Manhattan neighborhood rentals: [$0, $10000]
Price range for Entire Home rentals: [$0, $10000]
Room Type and Minimum Nights from Brooklyn Rentals:
             room_type  minimum_nights
5711   Entire home/apt               4
27104  Entire home/apt               3
1090      Private room               4
44066     Private room              30
46952  Entire home/apt               1
39976     Private room               2
20739  Entire home/apt               2
20485     Private room               2
38549  Entire home/apt              29
1419      Private room              14


# Identify Sensitive Information

Directions:

````
Based on the data analysis you did, identify the explicit identifiers, quasi-identifiers, sensitive attributes and non-sensitive attributes in the dataset. List these attributes with your reasons for your classification.
````

### Explicit Identifiers

* Id
    * **unique and explicit** index of property for rent

### Quasi-Identifiers

* Host_id
   * Could identify a property by knowing the host and other information.
* Name
    * Could identify a property based off the name given. 
* Host_name
    * Could identify a property by knowing the host and other information.
* Latitude
    * Location can help to identify a property.
* Longitude
    * Same as above.

### Sensitive Attributes

* Availability_365
    * Non-available dates can help reveal the use of the rental.
* Last_Review
    * Could indicate sensitive details about a property based on the last tenant's trip.
* Neighbourhood
    * Can reveal a lot about a certain rental property.
* Price
    * Reveals how valuable a property could be. Robbers would want to target richer properties.
* Calculated_host_listings_count
    * Reveals how many properties a person owns. 
* Number_of_reviews
    * Reveals popularity of a certain property.

### Non-sensitive Attributes

* Neighbourhood_group
    * Very vague information about location.
* Room_type
    * Not much information can be derived from this. This information could be inferred by location information.
* Reviews_per_month
    * Mostly reveals how often a property is used.
* Minimum_nights
    * Mostly reveals how much time the host is willing to accomodate tenants.

# Anonymization Approach

Develop a solution (coding) to anonymize the dataset. You can use any technique or combination of algorithms discussed in the class (k-anonymity, l-diversity, t-closeness, differential privacy, randomization,  cryptography, etc).

## Naive Anonymization

To start off any anonmyization it is good to first take the naive approach. By the naive approach I mean removing explicit identifiers as well as any quasi-identifiers that are not important to the utility of the data, or are likely to disclose too much information. To complete this first part I would remove the explicity identifier `id` as well as the quasi-identifiers `latitude` and `longitude`. I would remove these two pieces of location data, because the reveal the **exact** location of the property. This could lead to the disclosure of a lot of other information through publicly available auxillary data. Furthermore there already exists more vague, yet helpful location data in the form of neighbourhood and neighbourhood_group attributes.

### Example of QI being revealing

Let's look at the first row in the data set:

In [7]:
airbnb_data.head(1)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365


Just looking up the latitude and longitude in Google I am able to find this property's address: `805 Friel Pl, Brooklyn, NY 11218`. I can also see in the Whitepages to see that the host's full name is `John Layyapilli` in his 70s, and I can also see that most of the apartments in the building are used to rent out, since they are not listed as having residents. An attacker can easily leverage publicly available information to expose this record.

Going off this the first step in our anonymization process will be **removing these three columns**:

In [8]:
redacted_airbnb = airbnb_data.drop(['id','latitude','longitude'], axis=1)
redacted_airbnb.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,Private room,149,1,9,2018-10-19,0.21,6,365
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,Private room,150,3,0,NaN,NaN,1,365
3,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### Generalization and Suppression of Quasi-Identifiers

We can proceed by using *one-way-encryption* (hashing) to create unique integer identifiers for the `host_name` column. This approach maintains the uniqueness of each element while hiding the names of hosts.

We will then apply 42-anonymity to the `neighborhood` column.

We can also drop the `host_listings_count` column as it does not offer any utility to our aggregates.

In [9]:
redacted_airbnb["host_name"] = redacted_airbnb["host_name"].apply(lambda x: str(sha256(str(x).encode()).hexdigest()))
redacted_airbnb = redacted_airbnb.drop("calculated_host_listings_count", axis=1)

The K Anonymity implementation that we will use achieves its goal using the Datafly algorithm. The modules `datafly.py`, `tree.py` and `dgh.py` were written by [Alessio Vierti](https://github.com/alessiovierti/python-datafly).

Datafly, as discussed in class, is an algorithm which achieves a non-globally optimum K-Anonymity solution but is faster than other approaches.

It divides the quasi-identifiers into equivalence classes over their cartesian products, selects an attribute based on relative frequency against a "suppression threshold" and repeats this until less than k records are differentiable.

Though K-Anonymity is guaranteed, the algorithm suffers from some information loss due to its nature of being a greedy algorithm.

In [10]:
# First we save a copy of our current schema to modify with Datafly
with open("redacted.csv", mode='w', encoding='utf-8') as fileout:
    redacted_airbnb.to_csv(fileout, encoding='utf-8')

# Here we construct a subprocess call which invokes Datafly.
# We hand it our dataset, a list of quasi-identifier names, a set of hierarchy files, and a value for K.
run(["python", "datafly.py", "--private_table", "redacted.csv", "--quasi-identifier", "host_id, name, host_name, neighborhood", "--dgh" "FILES_GO_HERE", "-k" "42"], capture_output=True)

# Re-read the anonymized table.
with open("redacted.csv", mode='r', encoding='utf-8') as filein:
    redacted_airbnb = pd.read_csv(filein, encoding='utf-8')

### Differential Privacy

Our next step is to apply Laplacian noise in an effort to employ some differential privacy.
The implementation for the Laplace Mechanism came from [this StackOverflow Question](https://stackoverflow.com/q/40694682) as well as the extra credit material from class.

In [12]:
print(redacted_airbnb.head(5))
def laplaceMechanism(x, epsilon):
    x +=  np.random.laplace(0, 1.0/epsilon, 1)[0]
    return x

laplacecols = ["availability_365", "price", "number_of_reviews"]

for col in laplacecols:
    redacted_airbnb[col] = redacted_airbnb[col].apply(lambda x: laplaceMechanism(x, 0.25))

print(redacted_airbnb.head(5))

   Unnamed: 0                                              name  host_id  \
0           0                Clean & quiet apt home by the park     2787   
1           1                             Skylit Midtown Castle     2845   
2           2               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3           3                   Cozy Entire Floor of Brownstone     4869   
4           4  Entire Apt: Spacious Studio/Loft by central park     7192   

                                           host_name neighbourhood_group  \
0  a8cfcd74832004951b4408cdb0a5dbcd8c7e52d43f7fe2...            Brooklyn   
1  bda0a01c76aae97ef15d6b792f3dc3cbb6609d611e9c56...           Manhattan   
2  22c304f1b7815ee19c911b75c4a1be614630c44c5b70d1...           Manhattan   
3  2db6515fa9c139c4bb3808003f841f75adc9f9273d54e6...            Brooklyn   
4  f0b8649dbd8cc269a6a9f57166490602cb5e17344007e2...           Manhattan   

  neighbourhood        room_type       price  minimum_nights  \
0    Kensington     Pr

# Utility Loss